In [2]:
import cv2
import numpy as np


In [5]:
# Capture Training Data

cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")


skip = 0
face_data = []
dataset_path = './Data For Face Detection/'
file_name = input("Enter the name of the person : ")

while True:
    ret,frame = cap.read()
    
    if ret == False:
        continue
        
    #gray_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(frame,1.3,5)
    faces = sorted(faces,key=lambda f : f[2]*f[3])  # To get the largest face if there are multiple faces in the frame
    
    # Pick the last face (because it is the largest face according to area (f[2]*f[3]))
    for face in faces[-1:]:
        x,y,w,h = face
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
        
        # Extract (Crop out the face region only ) : region of Intreset
        
        offset = 10  # To get the extra area around the reactangle around face
        face_section = frame[y-offset:y+h+offset,x-offset:x+w+offset]
        
        # Resizing (or making the size of face same for all frames) the size of face
        face_section = cv2.resize(face_section,(100,100)) 
        
        if(skip%10==0):
            face_data.append(face_section)
            
    cv2.imshow("frame ",frame)
    cv2.imshow("face section ",face_section)
    
    key_pressed = cv2.waitKey(1) & 0xFF
    
    if key_pressed == ord('q'):
        break
        
        
face_data = np.asarray(face_data)
face_data = face_data.reshape(face_data.shape[0],-1)

#print(face_data.shape)

np.save(dataset_path+file_name+'.npy',face_data)

cap.release()
cv2.destroyAllWindows()
        

Enter the name of the person : Vishal


# Implemention of KNN 

In [3]:
import os

In [4]:
# function to calulate the diffence(Eucledian distance) between the faces on the basis of pixel intensity 

def distance(v1,v2):
    return np.sqrt(((v1-v2)**2).sum())

In [5]:
# KNN 

def knn(train_data, test_data,k=5):
    dist = []
    
    for i in range(train_data.shape[0]):
        ix = train_data[i,:-1]
        iy = train_data[i,-1]
        
        # compute the distance from test point
        d = distance(test_data,ix)
        dist.append([d,iy])
    # Sort based on the distance and get minimum k distance
    dk = sorted(dist,key=lambda x : x[0])[:k]

    # retrieve only the labels(Names of persons matching most)
    labels = np.array(dk)[:,-1]

    # Get frequencies of each label

    output = np.unique(labels,return_counts=True)

    # Find max frequnecy and corresnponding label
    index = np.argmax(output[1])

    return output[0][index]

In [6]:
# Face Detection
cap = cv2.VideoCapture(0)

face_cascade  = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')

skip = 0 
dataset_path = './Data For Face Detection/'

face_data = []
labels = []

class_id = 0 #Labels to map the given files of trained data
names = {}  # Mapping between id and name of person(file name)

# Data Preparation

for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        names[class_id] = fx[:-4]
        #print('loaded '+fx)
        data_item = np.load(dataset_path+fx)
        face_data.append(data_item)
        
        # Create labels for the class
        target = class_id*np.ones((data_item.shape[0],))
        class_id +=1
        labels.append(target)
        
face_dataset = np.concatenate(face_data,axis=0)
face_labels =  np.concatenate(labels,axis=0).reshape((-1,1))

#print(face_dataset.shape)
#print(face_labels.shape)

trainset = np.concatenate((face_dataset,face_labels),axis=1)

# print(trainset.shape)




# Tesing 

while True:
    ret,frame = cap.read()
    
    if ret == False:
        continue
        
    faces = face_cascade.detectMultiScale(frame,1.3,5)
    
    for face in faces:
        x,y,w,h = face
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
        
        # Extract (Crop out the face region only ) : region of Intreset
        offset = 10
        face_section = frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section = cv2.resize(face_section,(100,100))
        
        # Predicted Label (out)
        out = knn(trainset,face_section.flatten())
        
        #Display on the screen the name and rectangle around it
        pred_name = names[int(out)]
        cv2.putText(frame,pred_name,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
        
    cv2.imshow("Faces",frame)
    
    key = cv2.waitKey(1) & 0xFF
    if key==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()